# ###주의 이 파일은 worker, master를 연결만 하는 스크립트 입니다.
# worker, master에 대한 변경이 필요하면 terraform, userdata를 고쳐야 합니다.

# 순서!

## 1. terraform으로 클러스터 생성
## 2. user data로 들어가는 스크립트 파일로 kubernetes 설치
## 3. 설치 후 ending text 를 echo함
## 4. kube_join.txt 파일의 값을 읽어서 worker에 뿌림
## 5. 다 작성하면 쉘스크립트에 테라폼 apply 까지 묶어서 한번에 가자

# 정말 마지막 테스트 worker 3개.
## 아... 그냥 fully managed service 써도 되는 개발자들 부럽다...

# 근데 테라폼 한번 배워서 진짜 요긴하게 잘 써먹네.
## 솔직히 테라폼 없었으면 에러 한두번 발생했을때 인스턴스 다시 키는거 너무 힘들어서 좌절했을듯

https://askubuntu.com/questions/983351/how-to-install-terraform-in-ubuntu

!apt install -y wget
!wget https://releases.hashicorp.com/terraform/1.0.7/terraform_1.0.7_linux_amd64.zip
!unzip terraform_1.0.7_linux_amd64.zip
!mv terraform /usr/local/bin/
!terraform --version 

!rm terraform_1.0.7_linux_amd64.zip

In [1]:
!terraform init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/aws from the dependency lock file
- Reusing previous version of hashicorp/template from the dependency lock file
- Reusing previous version of hashicorp/http from the dependency lock file
- Using previously-installed hashicorp/template v2.2.0
- Using previously-installed hashicorp/http v3.3.0
- Using previously-installed hashicorp/aws v4.67.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [25]:
!terraform destroy -auto-approve

aws_vpc.kube_vpc: Refreshing state... [id=vpc-00ef9a845a70a873e]
aws_instance.kube_master: Refreshing state... [id=i-00da84a95a682fbfb]
aws_internet_gateway.kube_igw: Refreshing state... [id=igw-0086179b22bb83d98]
aws_subnet.public_subnet: Refreshing state... [id=subnet-0f4ac487eb0dcfc84]
aws_route_table.public_rt: Refreshing state... [id=rtb-0a49132c580bcaa79]
aws_security_group.sg_worker: Refreshing state... [id=sg-0c99ed3ef08387567]
aws_security_group.sg_master: Refreshing state... [id=sg-09bf254249028dc18]
aws_route.public_rt_igw: Refreshing state... [id=r-rtb-0a49132c580bcaa791080289494]
aws_launch_configuration.kube_worker_lc: Refreshing state... [id=terraform-20230525133602087600000001]
aws_route_table_association.public_rt_association: Refreshing state... [id=rtbassoc-0656b3fcfaf82923c]
aws_autoscaling_group.kube_worker_asg: Refreshing state... [id=terraform-20230525133603950800000003]

Note: Objects have changed outside of Terraform

Terraform detected the following changes ma

aws_route_table_association.public_rt_association: Destroying... [id=rtbassoc-0656b3fcfaf82923c]
aws_route.public_rt_igw: Destroying... [id=r-rtb-0a49132c580bcaa791080289494]
aws_autoscaling_group.kube_worker_asg: Destroying... [id=terraform-20230525133603950800000003]
aws_instance.kube_master: Destroying... [id=i-00da84a95a682fbfb]
aws_route_table_association.public_rt_association: Destruction complete after 1s
aws_route.public_rt_igw: Destruction complete after 1s
aws_internet_gateway.kube_igw: Destroying... [id=igw-0086179b22bb83d98]
aws_route_table.public_rt: Destroying... [id=rtb-0a49132c580bcaa79]
aws_route_table.public_rt: Destruction complete after 2s
aws_autoscaling_group.kube_worker_asg: Still destroying... [id=terraform-20230525133603950800000003, 10s elapsed]
aws_instance.kube_master: Still destroying... [id=i-00da84a95a682fbfb, 10s elapsed]
aws_internet_gateway.kube_igw: Still destroying... [id=igw-0086179b22bb83d98, 10s elapsed]
aws_instance.kube_master: Still destroying.

In [3]:
#!terraform plan

In [24]:
#!terraform apply -auto-approve

In [5]:
import time

print('wait for terraform')
time.sleep(30)

wait for terraform


## 만들어진 인스턴스 목록 획득

https://stackoverflow.com/questions/33297172/boto3-error-botocore-exceptions-nocredentialserror-unable-to-locate-credential

# 인스턴스 연결 설정 들

!mkdir -p ~/.aws
#!ls ~/.aws

!(echo " \
[default] \n \
aws_access_key_id = 키키키키 \n \
aws_secret_access_key = 쉬크릿쉬크릿쉬크릿쉬크릿 \
") | tee ~/.aws/credentials

!(echo " \
[default] \n \
region = us-east-1 \n \
output = json \
") | tee ~/.aws/config|

In [6]:
import boto3
import time

# 인스턴스가 생성되는 걸 기다렸다가 이름을 기준으로 master, worker 분류
## terraform에서는 현재 불가능 하거나, 두번에 나눠 해야 되서 여기서 진행

In [7]:
is_started = True

In [8]:
def get_cluster_instances():
    ec2 = boto3.resource('ec2')
    
    master_list = []
    worker_list = []
    
    for instance in ec2.instances.all():

        if instance.state['Name'] == 'pending' : 
            print("Wait for pending", instance )
            instance.wait_until_running()
            is_pending=True
            time.sleep(3)

        elif instance.state['Name'] == 'running' :
            for tag in instance.tags :
                if tag['Key'] == 'Name' and tag['Value'] == 'kube_master' :
                    print(instance, tag)
                    master_list.append(instance)
                if tag['Key'] == 'Name' and tag['Value'] == 'kube_worker' :
                    print(instance, tag)
                    worker_list.append(instance)

    return master_list, worker_list


## wait for pending이 있어서 사용했는데도
## 데이터가 처음 긁을땐 안가져와지는 현상이 있어 get again

In [9]:
master_list = []
worker_list = []
if is_started :
    print('wait for starting')
    time.sleep(10)
    is_started = False
    
master_list, worker_list = get_cluster_instances()
#이 코드 노드 숫자 늘어나면 분명히 문제 생길듯
#정 뭣하면 master 먼저 키고 다 된거 확인 한 후 worker 켜서 userdata에 넣어두던가...

#뭣하면 master_n, worker_n을 처음부터 써놓고 맞을때까지 기다리거나...
while (len(master_list) == 0) or (len(worker_list) == 0) :
    print("get again")
    master_list, worker_list = get_cluster_instances()

wait for starting
ec2.Instance(id='i-0472a33d932af30b4') {'Key': 'Name', 'Value': 'kube_worker'}
ec2.Instance(id='i-0996c6d6f13d8dea6') {'Key': 'Name', 'Value': 'kube_worker'}
ec2.Instance(id='i-02ab5225b7e636611') {'Key': 'Name', 'Value': 'kube_worker'}
ec2.Instance(id='i-00da84a95a682fbfb') {'Key': 'Name', 'Value': 'kube_master'}


In [10]:
master_list

[ec2.Instance(id='i-00da84a95a682fbfb')]

In [11]:
master_list[0].public_ip_address

'35.175.192.231'

In [12]:
worker_list[0].public_ip_address

'3.81.217.182'

In [13]:
worker_list

[ec2.Instance(id='i-0472a33d932af30b4'),
 ec2.Instance(id='i-0996c6d6f13d8dea6'),
 ec2.Instance(id='i-02ab5225b7e636611')]

In [14]:
import botocore
import paramiko
import time

#전형적인 template pattern이 필요한 사례같아서 한번 써봄
def exec_process(instance_ip, command_list, func_list):

    key = paramiko.RSAKey.from_private_key_file('DevOps_Study.pem')
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    print(instance_ip)
    result_list = []
    try:
        client.connect(hostname=instance_ip, username="ubuntu", pkey=key)

        for i in range(len(func_list)):
            result_list.append(
                func_list[i](instance_ip,command_list[i],client)
            )

        client.close()
        
    except Exception as e:
        print(e)
        client.close()
    finally:
        return result_list

def exec_command_func(instance_ip, command, client):
    # Execute a command(cmd) after connecting/ssh to an instance
    stdin, stdout, stderr = client.exec_command(command)
    
    output = str(stdout.read() , 'utf-8')
    
    if output[-1] == "\n":
        output = output[:-1]
    print(output)
    #print(stderr)
    
    if output == "kubernetes_ready\n" : #"end\n":#
        print(instance_ip, "is ready")
    return output

def check_in_while_loop(instance_ip, command, client):
    while(True) :
        # Execute a command(cmd) after connecting/ssh to an instance
        stdin, stdout, stderr = client.exec_command(command)
        
        output = str(stdout.read() , 'utf-8')
        
        if output[-1] == "\n":
            output = output[:-1]
        
        print(output)
        #print(stderr)
        
        if output == "kubernetes_ready" : #"end\n":#
            print(master_ip, "is ready")
            return True
        time.sleep(10)
            
    return False

# user data가 다 끝날때까지 대기 후 node 연결 정보 수집

In [15]:
def try_until_finish(master_ip):
    try :
        result_list = exec_process(instance_ip=master_ip, 
                                       command_list=['sudo tail -n 1 /var/log/user-data.log',
                                                     'sudo kubeadm token create',
                                                     "openssl x509 -pubkey -in /etc/kubernetes/pki/ca.crt | openssl rsa -pubin -outform der 2>/dev/null | openssl dgst -sha256 -hex | sed 's/^.* //'"
                                                    ], 
                                       func_list=[check_in_while_loop,
                                                  exec_command_func,
                                                  exec_command_func
                                                 ])
        return result_list
    except Exception as e:
        return try_until_finish(master_ip)

# 나중에는 여러개 노드를 써서 master node의 high availability를 충족해야 겠지만, 아직은.

In [16]:

for i in range(len(master_list)):

    """
    순서대로
    1. master 동작 완료 체크
    2. token 생성 (찾는거보다 생성이 편함... 혼자쓸때만인가?)
    3. --discovery-token-ca-cert-hash 찾기
    """
    
    print(i,'th job start')

    master_ip = master_list[i].public_ip_address
    result_list = try_until_finish(master_ip)
    """
    result_list = exec_process(instance_ip=master_ip, 
                                   command_list=['sudo tail -n 1 /var/log/user-data.log',
                                                 'sudo kubeadm token create',
                                                 "openssl x509 -pubkey -in /etc/kubernetes/pki/ca.crt | openssl rsa -pubin -outform der 2>/dev/null | openssl dgst -sha256 -hex | sed 's/^.* //'"
                                                ], 
                                   func_list=[check_in_while_loop,
                                              exec_command_func,
                                              exec_command_func
                                             ])
    """

0 th job start
35.175.192.231
Unpacking docker-ce (5:24.0.1-1~ubuntu.22.04~jammy) ...
No VM guests are running outdated hypervisor (qemu) binaries on this host.
done.
Unpacking kubernetes-cni (1.2.0-00) ...
No VM guests are running outdated hypervisor (qemu) binaries on this host.
[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'
[kubeconfig] Writing "admin.conf" kubeconfig file
[wait-control-plane] Waiting for the kubelet to boot up the control plane as static Pods from directory "/etc/kubernetes/manifests". This can take up to 4m0s
kubernetes_ready
35.175.192.231 is ready
8u56vs.o8jeqy3xonatt24j
bdc106eb6cb396d23bf5d20d0dfc20562ce079a98fd7c97bdeca8ef66f4d9240


In [17]:
result_list

[True,
 '8u56vs.o8jeqy3xonatt24j',
 'bdc106eb6cb396d23bf5d20d0dfc20562ce079a98fd7c97bdeca8ef66f4d9240']

## node 연결 명령어 작성

In [18]:
master_private = master_list[0].private_ip_address
token = result_list[1]
discovery_token_ca_cert_hash = result_list[2]

join_command = """sudo kubeadm join {}:6443 --token {} --discovery-token-ca-cert-hash sha256:{}""".format(master_private,
                                                                                                token,
                                                                                                discovery_token_ca_cert_hash
                                                                                               )

join_command

'sudo kubeadm join 10.1.0.119:6443 --token 8u56vs.o8jeqy3xonatt24j --discovery-token-ca-cert-hash sha256:bdc106eb6cb396d23bf5d20d0dfc20562ce079a98fd7c97bdeca8ef66f4d9240'

## worker 연결 : 수가 많아지면 multi thread를 써야 겠지만 아직은.

In [19]:
for i in range(len(worker_list)):
    
    print(i,'th job start')

    worker_ip = worker_list[i].public_ip_address
    result_list = exec_process(instance_ip=worker_ip, 
                               command_list=['sudo tail -n 1 /var/log/user-data.log',
                                             join_command
                                            ], 
                               func_list=[check_in_while_loop,
                                          exec_command_func
                                         ])

0 th job start
3.81.217.182
kubernetes_ready
35.175.192.231 is ready
[preflight] Running pre-flight checks
[preflight] Reading configuration from the cluster...
[preflight] FYI: You can look at this config file with 'kubectl -n kube-system get cm kubeadm-config -o yaml'
[kubelet-start] Writing kubelet configuration to file "/var/lib/kubelet/config.yaml"
[kubelet-start] Writing kubelet environment file with flags to file "/var/lib/kubelet/kubeadm-flags.env"
[kubelet-start] Starting the kubelet
[kubelet-start] Waiting for the kubelet to perform the TLS Bootstrap...

This node has joined the cluster:
* Certificate signing request was sent to apiserver and a response was received.
* The Kubelet was informed of the new secure connection details.

Run 'kubectl get nodes' on the control-plane to see this node join the cluster.

1 th job start
54.166.235.196
Unpacking kubelet (1.21.7-00) ...
Processing triggers for man-db (2.10.2-1) ...
kubernetes_ready
35.175.192.231 is ready
[preflight] Runn

In [20]:
for worker in worker_list:
    print(worker, worker.public_ip_address, worker.private_ip_address)

ec2.Instance(id='i-0472a33d932af30b4') 3.81.217.182 10.1.0.179
ec2.Instance(id='i-0996c6d6f13d8dea6') 54.166.235.196 10.1.0.4
ec2.Instance(id='i-02ab5225b7e636611') 3.95.233.235 10.1.0.230


## 결과 확인 : 여러개 하니까 나중에 넣은건 한동안 not ready 뜨기도 하네, 시간지나니까 health check끝나서 연결됨

In [21]:

    master_ip = master_list[0].public_ip_address
    result_list = exec_process(instance_ip=master_ip, 
                               command_list=["sudo kubectl get nodes"], 
                               func_list=[exec_command_func])

35.175.192.231
NAME            STATUS     ROLES                  AGE   VERSION
ip-10-1-0-119   Ready      control-plane,master   69s   v1.21.7
ip-10-1-0-179   Ready      <none>                 50s   v1.21.7
ip-10-1-0-230   NotReady   <none>                 7s    v1.21.7
ip-10-1-0-4     Ready      <none>                 18s   v1.21.7


In [22]:
for master in master_list:
    print(master, master.public_ip_address, master.private_ip_address)

ec2.Instance(id='i-00da84a95a682fbfb') 35.175.192.231 10.1.0.119


In [23]:
for worker in worker_list:
    print(worker, worker.public_ip_address, worker.private_ip_address)

ec2.Instance(id='i-0472a33d932af30b4') 3.81.217.182 10.1.0.179
ec2.Instance(id='i-0996c6d6f13d8dea6') 54.166.235.196 10.1.0.4
ec2.Instance(id='i-02ab5225b7e636611') 3.95.233.235 10.1.0.230
